In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

plt.style.use('fivethirtyeight')

import time


# due to Kaggle memory limitations and the enormous dataset size, a sample from the whole
# trainset will be used for ML modeling
train_sample_fraction = None


# another global variable that must be defined is the NA values rate / theshold to ommit columns with
# NA values that pass this rate
na_rate_threshold = 0.9

# if we want to avoid using a fraction of the train dataset then using the following variable will suffice
train_sample_num = 1500000

# theshold to remove columns with unbalanced features to their values 
unbalanced_feature_rate_threshold = 0.9

# Any results you write to the current directory are saved as output.


In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# to be deleted this cell
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_and_binary_columns = [c for c,v in dtypes.items() if v in numerics]


In [ ]:
%%time
train = pd.read_csv('../input/microsoft-malware-prediction/train.csv', dtype=dtypes)

In [ ]:
stats = []
for col in train.columns:
    stats.append((col, train[col].dtype, train[col].nunique(), train[col].isnull().sum() * 100 / train.shape[0], train[col].value_counts(normalize=True, dropna=False).values[0] * 100))
    
stats_df = pd.DataFrame(stats, columns=['Feature', "type", 'Unique_values', 'Percentage of missing values', 'Percentage of values in the biggest category'])

stats_df.sort_values('Percentage of missing values', ascending=False)

In [ ]:
good_cols = list(train.columns)

for col in train.columns:
    
    # remove columns with high NA rate
    na_rate = train[col].isnull().sum() / train.shape[0]
    
    # remove columns with high Unbalanced values rate
    unbalanced_rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    
    if na_rate > na_rate_threshold:
        good_cols.remove(col)
    elif unbalanced_rate > unbalanced_feature_rate_threshold:
        good_cols.remove(col)

In [ ]:
good_cols

In [ ]:
print("train data set reduced size in memory:", train.memory_usage().sum() / (1000000000))


In [ ]:
train = train[good_cols]

In [ ]:
del stats_df

In [ ]:
import gc

gc.collect()

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train['HasDetections'].value_counts()

In [ ]:
train['HasDetections'].value_counts().plot(kind="pie", figsize=(12,9), colormap="coolwarm")

In [ ]:
categorical_columns = list(train.loc[:, train.dtypes =="category"].columns)
numerical_and_binary_columns = list(train.loc[:, train.dtypes !="category"].columns)
numerical_columns = numerical_and_binary_columns

categorical_columns.remove("MachineIdentifier")

binary_columns = []
for col in (numerical_and_binary_columns):
    if train[col].nunique() == 2:
        binary_columns.append(col)
        numerical_columns.remove(col)

In [ ]:
categories_list = []
categories_list.append(len(categorical_columns))
categories_list.append(len(numerical_columns))
categories_list.append(len(binary_columns))

categories_df = pd.DataFrame(categories_list, 
                             index=["categorical_features", "numerical_features", "binary_features"])

categories_df = categories_df.transpose().plot(kind="barh", figsize=(21, 10), title="number of different features")

In [ ]:
if train_sample_fraction is not None:
    train_sample = train.sample(frac=train_sample_fraction, random_state=42)
elif train_sample_num is not None:
    train_sample = train.sample(n=train_sample_num, random_state=42)
else:
    train_sample = train.sample(n=1500000, random_state=42)

gc.collect()

In [ ]:
categorical_columns

In [ ]:
def categorical_univariate_and_bivariate_stats(feature):
    
    if feature in train_sample.columns:
    
        print("Top 10 most occurred categories for the categorical feature", feature)
        print(train_sample[feature].value_counts().head(10))

        f, axes = plt.subplots(1, 2, figsize=(21, 10))

        train_sample[feature].value_counts().head(10).plot.bar(ax=axes[0], colormap="BrBG")

        train_sample.groupby(["HasDetections", feature]).count()["MachineIdentifier"].unstack(0).sort_values(by=1, axis=0, ascending=False).head(10).plot.bar(ax=axes[1], colormap="coolwarm")
        
        f.suptitle("Categorical feature: "+" Univariate and Bivariate plots against the target variable")
        
    else:
        print("This feature has been removed from dataset due to high NaN rate or highly unbalanced values")
        
        
def logistic_fit(feature):
    
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import r2_score

    if feature in train_sample.columns:
        
        from sklearn.linear_model import LogisticRegression
        
        f, axes = plt.subplots(1, 2, figsize=(21, 10))

        # test if there is a logistic relationship between the feature1 and the target.
        print()
        print("Fitting a logistic regression model for the feature", feature,"against the target variable")
        
               
        mask = ~train_sample[feature].isnull() & ~train_sample["HasDetections"].isnull()

        logmodel = LogisticRegression(C=1e5, solver='lbfgs')
        
        if feature in categorical_columns:        
            logmodel.fit(train_sample[feature][mask].cat.codes.values.reshape(-1,1), train_sample["HasDetections"][mask])
            predictions = logmodel.predict(train_sample[feature][mask].cat.codes.values.reshape(-1,1))
        else:
            logmodel.fit(train_sample[feature][mask].values.reshape(-1,1), train_sample["HasDetections"][mask])
            predictions = logmodel.predict(train_sample[feature][mask].values.reshape(-1,1))

        from sklearn.metrics import classification_report
        print(classification_report(train_sample["HasDetections"][mask], predictions))
        print("")
        print("accuracy score:", accuracy_score(train_sample["HasDetections"][mask], predictions))
        print("F1 score:", accuracy_score(train_sample["HasDetections"][mask], predictions))
        #print("R^2 score:", r2_score(train_sample["HasDetections"][mask], predictions))

        import scikitplot as skplt
        skplt.metrics.plot_confusion_matrix(train_sample["HasDetections"][mask], predictions, normalize=False,
                                            title = "Confusion matrix for the feature: "+feature+" against the target variable after fitting a logistic regression model",
                                           figsize=(10,8), text_fontsize='medium', cmap="BrBG", ax = axes[0])
        
        
        # import statsmodels.api as sm
        # print()
        # est = sm.Logit(train_sample["HasDetections"][mask], train_sample[feature][mask].cat.codes.values.reshape(-1,1))
        # result1=est.fit()
        # print(result1.summary())
        if feature in categorical_columns:
            axes[1] = plt.scatter(train_sample[feature][mask].cat.codes.values.reshape(-1,1), predictions)
            axes[1] = plt.scatter(train_sample[feature][mask].cat.codes.values.reshape(-1,1), logmodel.predict_proba(train_sample[feature][mask].cat.codes.values.reshape(-1,1))[:,1])
            plt.xlabel(feature)
            plt.ylabel("HasDetections Probability")
            plt.title("Probability of Detecting a Malware vs the "+ feature)
            plt.show()
        else:
            axes[1] = plt.scatter(train_sample[feature][mask].values.reshape(-1,1), predictions)
            axes[1] = plt.scatter(train_sample[feature][mask].values.reshape(-1,1), logmodel.predict_proba(train_sample[feature][mask].values.reshape(-1,1))[:,1])
            plt.xlabel(feature)
            plt.ylabel("HasDetections Probability")
            plt.title("Probability of Detecting a Malware vs the "+ feature)
            plt.show()
        
    else:
        print("This feature has been removed from dataset due to high NaN rate or highly unbalanced values")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="ProductName")
logistic_fit(feature="ProductName")


In [ ]:
categorical_univariate_and_bivariate_stats(feature="EngineVersion")
logistic_fit(feature="EngineVersion")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="AppVersion")
logistic_fit(feature="AppVersion")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="AvSigVersion")
logistic_fit(feature="AvSigVersion")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Platform")
logistic_fit(feature="Platform")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Processor")
logistic_fit(feature="Processor")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="OsVer")
logistic_fit(feature="OsVer")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="OsPlatformSubRelease")
logistic_fit(feature="OsPlatformSubRelease")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="OsBuildLab")
logistic_fit(feature="OsBuildLab")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="SmartScreen")
logistic_fit(feature="SmartScreen")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_MDC2FormFactor")
logistic_fit(feature="Census_MDC2FormFactor")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_ChassisTypeName")
logistic_fit(feature="Census_ChassisTypeName")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_PowerPlatformRoleName")
logistic_fit(feature="Census_PowerPlatformRoleName")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_InternalBatteryType")
logistic_fit(feature="Census_InternalBatteryType")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSVersion")
logistic_fit(feature="Census_OSVersion")


In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSArchitecture")
logistic_fit(feature="Census_OSArchitecture")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSBranch")
logistic_fit(feature="Census_OSBranch")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSEdition")
logistic_fit(feature="Census_OSEdition")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSSkuName")
logistic_fit(feature="Census_OSSkuName")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSInstallTypeName")
logistic_fit(feature="Census_OSInstallTypeName")


In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_OSWUAutoUpdateOptionsName")
logistic_fit(feature="Census_OSWUAutoUpdateOptionsName")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_GenuineStateName")
logistic_fit(feature="Census_GenuineStateName")

In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_ActivationChannel")
logistic_fit(feature="Census_ActivationChannel")


In [ ]:
categorical_univariate_and_bivariate_stats(feature="Census_FlightRing")
logistic_fit(feature="Census_FlightRing")

In [ ]:
gc.collect()

In [ ]:
numerical_and_binary_columns

In [ ]:
def numerical_univariate_and_bivariate_plot(feature, num_of_bins = 40):
    
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})
    
    if feature in train_sample.columns:
    
        print("Top 10 Values counts for the numerical feature", feature)
        print(train_sample[feature].value_counts().head(10))
        print("Min value", train_sample[feature].min())
        print("Max value", train_sample[feature].max())
        print("NaN values", train_sample[feature].isnull().sum())
        print("Number of unique values", train_sample[feature].nunique())

        if train[feature].nunique() > 2:
            print("Mean value", train_sample[feature].mean())
            print("Variance value", train_sample[feature].var())

        # for binary features
        if train[feature].nunique() <= 2:

            f, axes = plt.subplots(1, 2, figsize=(21, 10))

            sns.countplot(x=feature, data=train_sample, ax=axes[0])
            sns.countplot(x=feature, hue = "HasDetections", data=train_sample, ax=axes[1], palette=['b','r'])
            
            f.suptitle("Numerical feature: "+feature+" Univariate and Bivariate plots against the target variable")

        # for numeric features
        else:

            f, axes = plt.subplots(1, 3, figsize=(21, 10))

            sns.distplot(train_sample[feature].dropna(), rug=False, kde=False, ax=axes[0], bins = num_of_bins)

            sns.violinplot(x="HasDetections", y = feature, hue="HasDetections", data=train_sample, ax=axes[1], palette=['b','r'])

            if feature == "LocaleEnglishNameIdentifier":
                sns.distplot(train_sample[train_sample["HasDetections"] == 0][feature].dropna().astype("int16"), rug=False, kde=False, color="b", ax=axes[2], bins = num_of_bins)
                sns.distplot(train_sample[train_sample["HasDetections"] == 1][feature].dropna().astype("int16"), rug=False, kde=False, color="r", ax=axes[2], bins = num_of_bins)
            else:
                sns.distplot(train_sample[train_sample["HasDetections"] == 0][feature].dropna(), rug=False, kde=False, color="b", ax=axes[2], bins = num_of_bins)
                sns.distplot(train_sample[train_sample["HasDetections"] == 1][feature].dropna(), rug=False, kde=False, color="r", ax=axes[2], bins = num_of_bins)
            
                f.suptitle("Numerical feature: "+feature+" Univariate and Bivariate plots against the target variable")
    else:
        print("This feature has been removed from dataset due to high NaN rate or highly unbalanced values")

In [ ]:
numerical_univariate_and_bivariate_plot(feature = "IsBeta")

In [ ]:
numerical_univariate_and_bivariate_plot(feature="RtpStateBitfield")
logistic_fit("RtpStateBitfield")

In [ ]:
numerical_univariate_and_bivariate_plot(feature="IsSxsPassiveMode")
logistic_fit("IsSxsPassiveMode")

In [ ]:
numerical_univariate_and_bivariate_plot(feature="AVProductStatesIdentifier")
logistic_fit("AVProductStatesIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot(feature="AVProductsInstalled")
logistic_fit("AVProductsInstalled")

In [ ]:
numerical_univariate_and_bivariate_plot("HasTpm")
logistic_fit("HasTpm")

In [ ]:
numerical_univariate_and_bivariate_plot("CountryIdentifier")
logistic_fit("CountryIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("CityIdentifier")
logistic_fit("CityIdentifier")


In [ ]:
numerical_univariate_and_bivariate_plot("OrganizationIdentifier")
logistic_fit("OrganizationIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("GeoNameIdentifier")
logistic_fit("GeoNameIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("LocaleEnglishNameIdentifier")
logistic_fit("LocaleEnglishNameIdentifier")


In [ ]:
numerical_univariate_and_bivariate_plot("OsBuild")
logistic_fit("OsBuild")

In [ ]:
numerical_univariate_and_bivariate_plot("OsSuite")
logistic_fit("OsSuite")

In [ ]:
numerical_univariate_and_bivariate_plot("IsProtected")
logistic_fit("IsProtected")

In [ ]:
numerical_univariate_and_bivariate_plot("AutoSampleOptIn")
logistic_fit("AutoSampleOptIn")

In [ ]:
numerical_univariate_and_bivariate_plot("SMode")
logistic_fit("SMode")

In [ ]:
numerical_univariate_and_bivariate_plot("IeVerIdentifier")
logistic_fit("IeVerIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Firewall")
logistic_fit("Firewall")

In [ ]:
numerical_univariate_and_bivariate_plot("UacLuaenable")
logistic_fit("UacLuaenable")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_OEMNameIdentifier")
logistic_fit("Census_OEMNameIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_OEMModelIdentifier")
logistic_fit("Census_OEMModelIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_ProcessorCoreCount")
logistic_fit("Census_ProcessorCoreCount")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_ProcessorManufacturerIdentifier")
logistic_fit("Census_ProcessorManufacturerIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_ProcessorModelIdentifier")
logistic_fit("Census_ProcessorModelIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_PrimaryDiskTotalCapacity")
logistic_fit("Census_PrimaryDiskTotalCapacity")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_SystemVolumeTotalCapacity")
logistic_fit("Census_SystemVolumeTotalCapacity")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_HasOpticalDiskDrive")
logistic_fit("Census_HasOpticalDiskDrive")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_TotalPhysicalRAM")
logistic_fit("Census_TotalPhysicalRAM")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_InternalPrimaryDiagonalDisplaySizeInInches")
logistic_fit("Census_InternalPrimaryDiagonalDisplaySizeInInches")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_InternalPrimaryDisplayResolutionHorizontal")
logistic_fit("Census_InternalPrimaryDisplayResolutionHorizontal")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_InternalPrimaryDisplayResolutionVertical")
logistic_fit("Census_InternalPrimaryDisplayResolutionVertical")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_InternalBatteryNumberOfCharges")
logistic_fit("Census_InternalBatteryNumberOfCharges")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_OSBuildNumber")
logistic_fit("Census_OSBuildNumber")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_OSBuildRevision")
logistic_fit("Census_OSBuildRevision")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_OSInstallLanguageIdentifier")
logistic_fit("Census_OSInstallLanguageIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_OSUILocaleIdentifier")
logistic_fit("Census_OSUILocaleIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsPortableOperatingSystem")
logistic_fit("Census_IsPortableOperatingSystem")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsFlightingInternal")
logistic_fit("Census_IsFlightingInternal")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsFlightsDisabled")
logistic_fit("Census_IsFlightsDisabled")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_ThresholdOptIn")
logistic_fit("Census_ThresholdOptIn")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_FirmwareManufacturerIdentifier")
logistic_fit("Census_FirmwareManufacturerIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_FirmwareManufacturerIdentifier")
logistic_fit("Census_FirmwareManufacturerIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_FirmwareVersionIdentifier")
logistic_fit("Census_FirmwareVersionIdentifier")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsSecureBootEnabled")
logistic_fit("Census_IsSecureBootEnabled")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsWIMBootEnabled")
logistic_fit("Census_IsWIMBootEnabled")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsVirtualDevice")
logistic_fit("Census_IsVirtualDevice")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsTouchEnabled")
logistic_fit("Census_IsTouchEnabled")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsPenCapable")
logistic_fit("Census_IsPenCapable")

In [ ]:
numerical_univariate_and_bivariate_plot("Census_IsAlwaysOnAlwaysConnectedCapable")
logistic_fit("Census_IsAlwaysOnAlwaysConnectedCapable")

In [ ]:
numerical_univariate_and_bivariate_plot("Wdft_IsGamer")
logistic_fit("Wdft_IsGamer")

In [ ]:
numerical_univariate_and_bivariate_plot("Wdft_RegionIdentifier")
logistic_fit("Wdft_RegionIdentifier")

In [ ]:
gc.collect()

In [ ]:
start_time = time.time()

sns.set(rc={'figure.figsize':(34.7,39.27)})

# Compute the correlation matrix
corr = train[numerical_and_binary_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm')

print("elapsed time:", time.time() - start_time, "seconds")


In [ ]:
s = corr.unstack().drop_duplicates()
so = s.sort_values(kind="quicksort")

print("Top most highly megative correlated features:")
print(so[(so<-0.4)])
print()

print("Top most highly positive correlated features:")
print(so[(so<1) & (so>0.5)].sort_values(ascending=False))

In [ ]:
del corr
del mask
del s
del so
gc.collect()

In [ ]:
%%time
def multivariate_plot(feature1, feature2):
    
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})
    
    # check if one of the two features exists in train_sample columns
    if (feature1 not in train_sample.columns) | (feature2 not in train_sample.columns):
        if (feature1 not in train_sample.columns):
            print(feature1, "has been removed due to high NA rate or highly unbalanced values")
        elif (feature2 not in train_sample.columns):
            print(feature2, "has been removed due to high NA rate or higlhy unbalanced values")
        print()
            
    else:
        
        f, axes = plt.subplots(1, 2, figsize=(21, 10))
        
        # Due to overflow error I have to convert some columns into a bigger data type
        if (feature1 == "LocaleEnglishNameIdentifier") & (feature2 == "Census_OSUILocaleIdentifier"):
            g1 = sns.scatterplot(x=train_sample["LocaleEnglishNameIdentifier"].astype('int16'), y=train_sample["Census_OSUILocaleIdentifier"], hue = train_sample["HasDetections"], ax = axes[0])
            #g1 = sns.jointplot(x=train_sample["LocaleEnglishNameIdentifier"].astype('int16'), y=train_sample["Census_OSUILocaleIdentifier"], height=9, data = train)
            
            plt.title('Scatterplot Between: '+feature1+" and "+feature2, y=1.3, fontsize = 9)
            
        elif (feature1 == "LocaleEnglishNameIdentifier") & (feature2 == "Census_OSInstallLanguageIdentifier"):
            g1 = sns.scatterplot(x=train_sample["LocaleEnglishNameIdentifier"].astype('float16'), y=train_sample["Census_OSInstallLanguageIdentifier"], hue = train_sample["HasDetections"], ax = axes[0])
            #g1 = sns.jointplot(x=train_sample["LocaleEnglishNameIdentifier"].astype('float16'), y=train_sample["Census_OSInstallLanguageIdentifier"], height=9, data = train)
            
            plt.title('Scatterplot Between: '+feature1+" and "+feature2, y=1.3, fontsize = 9)
        else:    
            g1 = sns.scatterplot(x=train_sample[feature1], y=train_sample[feature2], hue = train_sample["HasDetections"], ax = axes[0])
            
            axes[0].set_title('Scatterplot Between: '+feature1+" and "+feature2)

        #g2 = sns.lmplot(x=feature1, y=feature2, hue="HasDetections", n_boot=1, data = train_sample, height=9, markers=["o", "x"], palette=["b", "r"])
        #g2 = sns.jointplot(x=feature1, y=feature2, data = train_sample, hue = "HasDetections", height=9)
        #g2 = sns.scatterplot(x=feature1, y=feature2, data = train_sample, hue = "HasDetections")
        
        #plt.title('Scatterplot Between: '+feature1+" and "+feature2+" against the target variable: 'HasDetections'", y=1.1, fontsize = 16)
        
        
        # fitting a logistic regression model with the 2 features
        
        # test if there is a logistic relationship between the feature1 and the target.
        
        from sklearn.metrics import accuracy_score
        from sklearn.metrics import precision_score
        from sklearn.metrics import recall_score
        from sklearn.metrics import f1_score
        from sklearn.metrics import r2_score
        
        from sklearn.linear_model import LogisticRegression
        
        print()
        print("Fitting a logistic regression model for the features", feature1,"and",feature2,"against the target variable")
        
        mask = ~train_sample[feature1].isnull() & ~train_sample[feature2].isnull() & ~train_sample["HasDetections"].isnull()

        logmodel = LogisticRegression()
        
        logmodel.fit(train_sample[[feature1, feature2]][mask].values, train_sample["HasDetections"][mask])
        predictions = logmodel.predict(train_sample[[feature1, feature2]][mask].values)

        from sklearn.metrics import classification_report
        print(classification_report(train_sample["HasDetections"][mask], predictions))
        print("")
        print("accuracy score:", accuracy_score(train_sample["HasDetections"][mask], predictions))
        print("F1 score:", accuracy_score(train_sample["HasDetections"][mask], predictions))
        #print("R^2 score:", r2_score(train_sample["HasDetections"][mask], predictions))
        
        
        g2 = sns.scatterplot(x=train_sample[feature1][mask].values, y=train_sample[feature2][mask].values, hue = predictions, ax=axes[1])
        axes[1].set_title('Logistic Predictions Between: '+feature1+" and "+feature2)

        import scikitplot as skplt
        skplt.metrics.plot_confusion_matrix(train_sample["HasDetections"][mask], predictions, normalize=False,
                                            title = "Confusion matrix for two features: "+feature1+" and "+feature2+" against the target variable after fitting a logistic regression model",
                                           figsize=(10,8), text_fontsize='large', cmap="BrBG")
        

        #### my mistake, how can I fit a linear model for binary classification, I have to use here logistic regression as before :D
        
        # old code in comments
        '''
        ### test if a linear model will fit the 2 correlated feature against the target variable
        # my thanks to: https://www.datasciencecentral.com/profiles/blogs/linear-regression-in-python-use-of-numpy-scipy-and-statsmodels
        # this link helped me a lot https://stackoverflow.com/questions/13643363/linear-regression-of-arrays-containing-nans-in-python-numpy
        
        
        # test if there is a linear relationship between the feature1 and the target.
        from scipy import stats
        mask = ~np.isnan(train_sample[feature1]) & ~np.isnan(train_sample["HasDetections"])
        slope, intercept, r_value, p_value, std_err = stats.linregress(train_sample[feature1][mask], train_sample["HasDetections"][mask])

        sign='+'
        if slope<0:
            sign = ''
        
        lm_1 = "linear regression function #1: y = "+str(np.round(intercept,5))+" "+sign +" "+str(np.round(slope,5))+" * "+feature1
        lm_1 = lm_1+"\n"+" p-value: "+str(p_value)+"\n"+" R^2: "+str(r_value)
        #print(lm_1)
        #print(feature1, "p-value",p_value)
        #print("R^2:", r_value)

        #print()

        # test if there is a linear relationship between the feature2 and the target.
        mask = ~np.isnan(train_sample[feature2]) & ~np.isnan(train_sample["HasDetections"])
        slope, intercept, r_value, p_value, std_err = stats.linregress(train_sample[feature2][mask], train_sample["HasDetections"][mask])

        sign='+'
        if slope<0:
            sign=''
        
        lm_2 = "linear regression function #2: y = "+str(np.round(intercept,5))+" "+sign +" "+str(np.round(slope,5))+" * "+feature2
        lm_2 = lm_2+"\n"+" p-value: "+str(p_value)+"\n"+" R^2: "+str(r_value)
        #print(lm_2)
        #print(feature2, "p-value",p_value)
        #print("R^2:", 100* r_value)
        
        plt.figure(figsize=(8, 9))
        ax = plt.subplot(121)
    
        # plotting text with matplotlib:
        # https://matplotlib.org/api/_as_gen/matplotlib.pyplot.text.html
        plt.text(0.5, 0.9, lm_1, size=15, ha="center", va="center",
         bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   )
         )
        
        
        plt.text(0.5, 0.6, lm_2, size=15, ha="center", va="center",
         bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   )
         )

        plt.axis('off')
        plt.show()
        '''

In [ ]:
%%time
multivariate_plot("RtpStateBitfield", "IsSxsPassiveMode")
gc.collect()

In [ ]:
%%time
multivariate_plot("AVProductStatesIdentifier", "AVProductsInstalled")
gc.collect()

In [ ]:
%%time
multivariate_plot("Census_OSBuildNumber", "Census_OSBuildRevision")
gc.collect()

In [ ]:
%%time
multivariate_plot("OsBuild", "Census_OSBuildRevision")
gc.collect()

In [ ]:
%%time
multivariate_plot("HasTpm", "IeVerIdentifier")
gc.collect()

In [ ]:
%%time
multivariate_plot("LocaleEnglishNameIdentifier", "Census_OSUILocaleIdentifier")
gc.collect()

In [ ]:
%%time
multivariate_plot("Census_InternalBatteryNumberOfCharges", "Census_FirmwareManufacturerIdentifier")
gc.collect()

In [ ]:
%%time
multivariate_plot("LocaleEnglishNameIdentifier", "Census_OSInstallLanguageIdentifier")
gc.collect()

In [ ]:
%%time
multivariate_plot("Census_OEMModelIdentifier", "Census_FirmwareManufacturerIdentifier")
gc.collect()

In [ ]:
%%time
multivariate_plot("Census_OEMNameIdentifier", "Census_FirmwareVersionIdentifier")
gc.collect()

In [ ]:
del train
gc.collect()

In [ ]:
test_dtypes = {k: v for k, v in dtypes.items() if k in good_cols}

# get all columns except
test = pd.read_csv('../input/microsoft-malware-prediction/test.csv', dtype=test_dtypes, usecols=good_cols[:-1])

#test = reduce_mem_usage(test

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
new = train_sample["OsBuildLab"].str.split(".", expand = True)
train_sample["OsBuildLab_1"] = new[2]

new = test["OsBuildLab"].str.split(".", expand = True)
test["OsBuildLab_1"] = new[2]


categorical_columns.append("OsBuildLab_1")
del new

#special thanks to https://www.kaggle.com/delayedkarma/let-s-add-some-new-features-lb-0-674

#train_sample['new_num_1'] = train_sample['Census_TotalPhysicalRAM'] * train_sample['Census_InternalPrimaryDiagonalDisplaySizeInInches']
#test['new_num_1'] = test['Census_TotalPhysicalRAM'] * test['Census_InternalPrimaryDiagonalDisplaySizeInInches']

#train_sample['new_num_2'] = train_sample['Census_ProcessorCoreCount'] * train_sample['Census_InternalPrimaryDiagonalDisplaySizeInInches']
#test['new_num_2'] = test['Census_ProcessorCoreCount'] * test['Census_InternalPrimaryDiagonalDisplaySizeInInches']

#train_sample['new_num_3'] = train_sample['Census_ProcessorCoreCount'] * train_sample['Census_TotalPhysicalRAM']
#test['new_num_3'] = test['Census_ProcessorCoreCount'] * test['Census_TotalPhysicalRAM']

#train_sample['new_num_4'] = train_sample['Census_PrimaryDiskTotalCapacity'] * train_sample['Census_TotalPhysicalRAM']
#test['new_num_4'] = test['Census_PrimaryDiskTotalCapacity'] * test['Census_TotalPhysicalRAM']

#train_sample['new_num_5'] = train_sample['Census_SystemVolumeTotalCapacity'] * train_sample['Census_InternalPrimaryDiagonalDisplaySizeInInches']
#test['new_num_5'] = test['Census_SystemVolumeTotalCapacity'] * test['Census_InternalPrimaryDiagonalDisplaySizeInInches']

In [ ]:
train_sample = train_sample.drop(['MachineIdentifier'], axis=1)
test = test.drop(['MachineIdentifier'], axis=1)

In [ ]:
train_sample = train_sample.reset_index(drop=True)

In [ ]:
modes = train_sample.mode()

for col in train_sample.columns:
    train_sample[col] = np.where(train_sample[col].isnull(), modes[col], train_sample[col])

del modes

In [ ]:
modes_test = test.mode()

for col in test.columns:
    test[col] = np.where(test[col].isnull(), modes_test[col], test[col])

#train_sample.shape
del modes_test

In [ ]:
train_shape = train_sample.shape
test_shape = test.shape

train_and_test = pd.concat([train_sample,test], axis="rows", sort=False)

del train_sample
del test
gc.collect()


In [ ]:
train_and_test.head()

In [ ]:
train_and_test.tail()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:
        #print(i)
        labelencoder_X=LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])

MultiLabelEncoder(categorical_columns, train_and_test)
gc.collect()

In [ ]:
train_sample = train_and_test[0:train_shape[0]]
test = train_and_test[(train_shape[0]):(train_and_test.shape[0]+1)]

In [ ]:
del train_and_test

In [ ]:
test = test.drop(["HasDetections"], axis = 1)

In [ ]:
y = train_sample['HasDetections']
X = train_sample.drop(['HasDetections'], axis=1)

In [ ]:
del train_sample
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import time

# create a 70/30 split of the data 
xtrain, xvalid, ytrain, yvalid = train_test_split(X, y, random_state=42, test_size=0.3)

import xgboost as xgb

start_time = time.time()

# special thanks to https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
# these parameters have been found via xgboost tuning, you can see my tries in the commented tuning python snippets below:
# unfortunately it takes so much time to tune and to produce the final optimal classifier due to 9h time limit in Kaggle kernels.
clf_xgb = xgb.XGBClassifier(learning_rate=0.03, 
                            n_estimators=1000, 
                            max_depth=11,
                            min_child_weight=9,
                            gamma=0.2,
                            subsample=1,
                            colsample_bytree=0.4,
                            objective= 'binary:logistic',
                            nthread=-1,
                            scale_pos_weight=1,
                            reg_alpha = 0.6,
                            reg_lambda = 3,
                            seed=42)

clf_xgb.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xvalid, yvalid)], 
            early_stopping_rounds=100, eval_metric='auc', verbose=100)

predictions = clf_xgb.predict(xvalid)

print()
print(classification_report(yvalid, predictions))

print()
print("accuracy_score", accuracy_score(yvalid, predictions))

print()
predictions_probas = clf_xgb.predict_proba(xvalid)
print("roc-auc score for the class 1, from target 'HasDetections' ", roc_auc_score(yvalid, predictions_probas[:,1]))

print()
print("elapsed time in seconds: ", time.time() - start_time)

print()
gc.collect()

In [ ]:
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_confusion_matrix(yvalid, predictions, cmap="BrBG")

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_roc(yvalid, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_ks_statistic(yvalid, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_precision_recall(yvalid, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_cumulative_gain(yvalid, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(8,8)})
skplt.metrics.plot_lift_curve(yvalid, predictions_probas)

In [ ]:
sns.set(rc={'figure.figsize':(12, 18)})
xgb.plot_importance(clf_xgb, title='Feature importance', xlabel='F score', ylabel='Features')

In [ ]:
gc.collect()

In [ ]:
del X
del y
del xvalid
del yvalid
del xtrain
del ytrain
del predictions
del predictions_probas

In [ ]:
gc.collect()

In [ ]:
predictions_proba_test_list = []

chunck = 400000
test_times = test.shape[0] // chunck
test_rest = test.shape[0] % chunck

for i in  np.arange(0,(chunck * (test_times+1)), chunck):
    predictions_proba_test = list(clf_xgb.predict_proba(test[i:(i+chunck)])[:,1])
    predictions_proba_test_list.append(predictions_proba_test)
    #print("times:", i)


# flatten the list of lists
predictions_proba_test_list = [y for x in predictions_proba_test_list for y in x]

print(np.shape(predictions_proba_test_list))
print(test.shape)
gc.collect()

In [ ]:
del test

In [ ]:
del clf_xgb

In [ ]:
submission = pd.read_csv('../input/microsoft-malware-prediction/sample_submission.csv')
submission['HasDetections'] = predictions_proba_test_list
submission.to_csv('xgboost.csv', index=False)